# Music Recommender System using ALS Algorithm with Apache Spark and Python
+ **Estimated Execution Time (whole script): 2 minutes**
+ **Estimated Time (to complete the project): 8 hours**

## Description

For this project, you are to create a recommender system that will recommend new musical artists to a user based on their listening history. Suggesting different songs or musical artists to a user is important to many music streaming services, such as Pandora and Spotify. In addition, this type of recommender system could also be used as a means of suggesting TV shows or movies to a user (e.g., Netflix). 

To create this system you will be using Spark and the collaborative filtering technique. The instructions for completing this project will be laid out entirely in this file. You will have to implement any missing code as well as answer any questions.

**Submission Instructions:** 
* Add all of your updates to this Jupyter Notebook file and do NOT clear any of the output you get from running your code.
* Upload this file and the genererated HTML onto Moodle as a single zip folder called with your user name.

## Datasets

You will be using some publicly available song data from audioscrobbler, which can be found [here](http://www-etud.iro.umontreal.ca/~bergstrj/audioscrobbler_data.html). However, we modified the original data files so that the code will run in a reasonable time on a single machine. The reduced data files have been suffixed with `_small.txt` and contains only the information relevant to the top 50 most prolific users (highest artist play counts).

The original data file `user_artist_data.txt` contained about 141,000 unique users, and 1.6 million unique artists. About 24.2 million users’ plays of artists are recorded, along with their count.

Note that when plays are scribbled, the client application submits the name of the artist being played. This name could be misspelled or nonstandard, and this may only be detected later. For example, "The Smiths", "Smiths, The", and "the smiths" may appear as distinct artist IDs in the data set, even though they clearly refer to the same artist. So, the data set includes `artist_alias.txt`, which maps artist IDs that are known misspellings or variants to the canonical ID of that artist.

The `artist_data.txt` file then provides a map from the canonical artist ID to the name of the artist.

In [1]:
# !pip install findspark
# pip install --upgrade pyspark
# pip install https://github.com/pyinstaller/pyinstaller/archive/develop.tar.gz

In [2]:
# Import libraries
import findspark
findspark.init()

from pyspark.mllib.recommendation import *
import random
from operator import *
from collections import defaultdict

from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

In [3]:
# Initialize Spark Context
# YOUR CODE GOES HERE
from pyspark import SparkContext
sc = SparkContext(appName = "MusicRecommenderSystem")

## Loading data

Load the three datasets into RDDs and name them `artistData`, `artistAlias`, and `userArtistData`. View the README, or the files themselves, to see how this data is formated. Some of the files have tab delimeters while some have space delimiters. Make sure that your `userArtistData` RDD contains only the canonical artist IDs.

In [4]:
# Import test files from location into RDD variables
# YOUR CODE GOES HERE
artistData = sc.textFile("./data_raw/data_raw/artist_data_small.txt").map(lambda line: line.split("\t"))
artistAlias = sc.textFile("./data_raw/data_raw/artist_alias_small.txt").map(lambda line: line.replace('\t', ' '))
userArtistData = sc.textFile("./data_raw/data_raw/user_artist_data_small.txt")

## Data Exploration

In the blank below, write some code that with find the users' total play counts. Find the three users with the highest number of total play counts (sum of all counters) and print the user ID, the total play count, and the mean play count (average number of times a user played an artist). Your output should look as follows:
```
User 1059637 has a total play count of 674412 and a mean play count of 1878.
User 2064012 has a total play count of 548427 and a mean play count of 9455.
User 2069337 has a total play count of 393515 and a mean play count of 1519.
```



In [5]:
# Split a sequence into seperate entities and store as int
# YOUR CODE GOES HERE
userArtistData = userArtistData.map(lambda line: list(map(int,line.split())))

# Create a dictionary of the 'artistAlias' dataset
# YOUR CODE GOES HERE
artistAliasDict = artistAlias.map(lambda line: map(int,line.split())).collectAsMap() #key is bad id, value is good id

# If artistid exists, replace with artistsid from artistAlias, else retain original
# YOUR CODE GOES HERE
userArtistData = userArtistData.map(lambda entity: (entity[0], artistAliasDict[entity[1]] if entity[1] in artistAliasDict else entity[1], entity[2]))

# Create an RDD consisting of 'userid' and 'playcount' objects of original tuple
# YOUR CODE GOES HERE
songs_per_user = userArtistData.map(lambda entity:(entity[0],entity[2])).reduceByKey(lambda a,b: a+b)

# Count instances by key and store in broadcast variable
# YOUR CODE GOES HERE
# artist_count_per_user = userArtistData.map(lambda entity:(entity[0],entity[1]))\
# .groupByKey().mapValues(lambda values: len(set(values)))  -- WE DON'T WANT UNIQUE... WE WANT AS IT IS..

artist_count_per_user = userArtistData.map(lambda entity:(entity[0],1))\
.reduceByKey(lambda a,b:a+b) 

# Compute and display users with the highest playcount along with their mean playcount across artists
# YOUR CODE GOES HERE
users_songs_artists_counts = songs_per_user.join(artist_count_per_user)
final_results = users_songs_artists_counts.map(lambda record: (record[0], record[1][0],int((record[1][0]/record[1][1]))))

top_three_results = final_results.sortBy(lambda entity: entity[1],ascending = False).take(3)

for r in top_three_results:
    print("User {} has a total play count of {} and a mean play count of {}".format(r[0],r[1],r[2]))

User 1059637 has a total play count of 674412 and a mean play count of 1878
User 2064012 has a total play count of 548427 and a mean play count of 9455
User 2069337 has a total play count of 393515 and a mean play count of 1519


####  Splitting Data for Testing

Use the [randomSplit](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.randomSplit) function to divide the data (`userArtistData`) into:
* A training set, `trainData`, that will be used to train the model. This set should constitute 40% of the data.
* A validation set, `validationData`, used to perform parameter tuning. This set should constitute 40% of the data.
* A test set, `testData`, used for a final evaluation of the model. This set should constitute 20% of the data.

Use a random seed value of 13. Since these datasets will be repeatedly used you will probably want to persist them in memory using the [cache](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.cache) function.

In addition, print out the first 3 elements of each set as well as their sizes; if you created these sets correctly, your output should look like the following:
```
[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000114, 2)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000123, 2)]
[(1059637, 1000094, 1), (1059637, 1000112, 423), (1059637, 1000113, 5)]
19761
19862
9858
```

In [6]:
# Split the 'userArtistData' dataset into training, validation and test datasets. Store in cache for frequent access
# YOUR CODE GOES HERE
trainData,validationData,testData = userArtistData.randomSplit([0.4,0.4,0.2],seed=13)
trainData.cache()
validationData.cache()
testData.cache()

# Display the first 3 records of each dataset followed by the total count of records for each datasets
# YOUR CODE GOES HERE
print(trainData.take(3))
print(validationData.take(3))
print(testData.take(3))
print(trainData.count())
print(validationData.count())
print(testData.count())

[(1059637, 1000049, 1), (1059637, 1000056, 1), (1059637, 1000114, 2)]
[(1059637, 1000010, 238), (1059637, 1000062, 11), (1059637, 1000123, 2)]
[(1059637, 1000094, 1), (1059637, 1000112, 423), (1059637, 1000113, 5)]
19769
19690
10022


## The Recommender Model

For this project, we will train the model with implicit feedback. You can read more information about this from the collaborative filtering page: [http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html). The [function you will be using](http://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.ALS.trainImplicit) has a few tunable parameters that will affect how the model is built. Therefore, to get the best model, we will do a small parameter sweep and choose the model that performs the best on the validation set

Therefore, we must first devise a way to evaluate models. Once we have a method for evaluation, we can run a parameter sweep, evaluate each combination of parameters on the validation data, and choose the optimal set of parameters. The parameters then can be used to make predictions on the test data.

### Model Evaluation

Although there may be several ways to evaluate a model, we will use a simple method here. Suppose we have a model and some dataset of *true* artist plays for a set of users. This model can be used to predict the top X artist recommendations for a user and these recommendations can be compared the artists that the user actually listened to (here, X will be the number of artists in the dataset of *true* artist plays). Then, the fraction of overlap between the top X predictions of the model and the X artists that the user actually listened to can be calculated. This process can be repeated for all users and an average value returned.

For example, suppose a model predicted [1,2,4,8] as the top X=4 artists for a user. Suppose, that user actually listened to the artists [1,3,7,8]. Then, for this user, the model would have a score of 2/4=0.5. To get the overall score, this would be performed for all users, with the average returned.

**NOTE: when using the model to predict the top-X artists for a user, do not include the artists listed with that user in the training data.**

Name your function `modelEval` and have it take a model (the output of ALS.trainImplicit) and a dataset as input. For parameter tuning, the dataset parameter should be set to the validation data (`validationData`). After parameter tuning, the model can be evaluated on the test data (`testData`).

##### Notes for self
1. allArtists is the list of all artists there are. 
2. allUsers are the list of allUsers we have on our Validation Data.
3. We created two dictionaries for this user --- the list of their preferred artists from training data, the list of their preferred artists from validation data.
4. The model has been trained on training data, with user id, artist id and # of times listed (substitute for rating) as the columns. 
5. Prediction happens on validation data. Onlu user id and artist id are the inputs. Rating will be the response var. BUT, INSTEAD OF PREDICTING SCORES ON JUST THE VALIDATION SET OF ARTISTS FOR THE USER, WE ARE PREDICTING FOR ALL ARTISTS AND HOW THIS USER WILL SCORE THEM. 
6. Step 5 will require creating a prediction dataset with ALL USERS (of Validation Data) AND ALL ARTISTS for these users.
7. Predict for allUsersArtistsMapping and get the top n artists apart from those already in training data for each user in the following for-loop n users. 

In [7]:
def modelEval(model, dataset):
    
    # All artists in the 'userArtistData' dataset
    # YOUR CODE GOES HERE
    allArtists = sc.parallelize(set(userArtistData.map(lambda record: record[1]).collect()))
#     allArtists = sc.broadcast(allArtists.collect())
    
    
    # Set of all users in the current (Validation/Testing) dataset
    # YOUR CODE GOES HERE
    allUsers = sc.parallelize(set(dataset.map(lambda record: record[0]).collect()))
    
    
    # Create a dictionary of (key, values) for current (Validation/Testing) dataset
    # YOUR CODE GOES HERE
    currentDict = {}
    trainDict = {}
    for user in allUsers.collect(): #These are all users of the validation Data. We are getting their 'listened to' artists from current data
        artistsOfUserInCurrent = list(dataset.filter(lambda record: record[0] == user).map(lambda record:record[1]).collect())
        currentDict[user] = artistsOfUserInCurrent
    
    # Create a dictionary of (key, values) for training dataset
    # YOUR CODE GOES HERE
        artistsOfUserInTrain = list(trainData.filter(lambda record: record[0] == user).map(lambda record:record[1]).collect())
        trainDict[user] = artistsOfUserInTrain
    
    # For each user, calculate the prediction score i.e. similarity between predicted and actual artists
    # YOUR CODE GOES HERE
    userScores = []
    
    #For every user create an RDD of tuples mapping all artists to this user. This will become our data to predict on.
    allUsersArtistsMapping = allUsers.cartesian(allArtists)

    #Predict ratings and split the results as shown.
    predictions = model.predictAll(allUsersArtistsMapping).map(lambda res: (res[0], res[1], res[2])) #User, Product, Rating
    
    #For every user, we eliminate the predictions for Artists who were already seen in train data. 
    #Then we sort them in descending order of ratings to pick the top N recommended artists.
    for user in allUsers.collect():
        predictsForUser = predictions.filter(lambda res: res[0] == user and not res[1] in trainDict[user]).sortBy(lambda res: res[2],ascending = False)
        
         #Get top n artists as recommended by the validation set of artists for user.
        topNArtistsForUser = predictsForUser.map(lambda rec: rec[1]).take(len(currentDict[user]))
        
        #Check how many artist IDs overlap. they will be the score for the user.       
        userScore = len(set(topNArtistsForUser).intersection(set(currentDict[user])))/len(currentDict[user])
        userScores.append(userScore)

    # Print average score of the model for all users for the specified rank
    # YOUR CODE GOES HERE
    print("The model score for rank {} is {}".format(model.rank,sum(userScores)/len(userScores)))
#     return userScores

### Model Construction

Now we can build the best model possibly using the validation set of data and the `modelEval` function. Although, there are a few parameters we could optimize, for the sake of time, we will just try a few different values for the [rank parameter](http://spark.apache.org/docs/latest/mllib-collaborative-filtering.html#collaborative-filtering) (leave everything else at its default value, **except make `seed`=345**). Loop through the values [2, 10, 20] and figure out which one produces the highest scored based on your model evaluation function.

Note: this procedure may take several minutes to run.

For each rank value, print out the output of the `modelEval` function for that model. Your output should look as follows:
```
The model score for rank 2 is ~0.090431
The model score for rank 10 is ~0.095294
The model score for rank 20 is ~0.090248
```
Step below takes 2 minutes to run. Uncomment to if you wish to run and calculate model score. 

In [8]:
# rankList = [2,10,20]
# for rank in rankList:
#     model = ALS.trainImplicit(trainData, rank , seed=345)
#     modelEval(model,validationData)

In [9]:
bestModel = ALS.trainImplicit(trainData, rank=10, seed=345)
modelEval(bestModel, testData)

The model score for rank 10 is 0.06028154355485621


[0.06382978723404255,
 0.03289473684210526,
 0.10526315789473684,
 0.09283551967709384,
 0.014321819713563605,
 0.15269461077844312,
 0.1,
 0.0,
 0.06593406593406594,
 0.0379746835443038,
 0.06930693069306931,
 0.06818181818181818,
 0.10426540284360189,
 0.0,
 0.15286624203821655,
 0.0,
 0.0447427293064877,
 0.0,
 0.021739130434782608,
 0.055299539170506916,
 0.02127659574468085,
 0.025423728813559324,
 0.07407407407407407,
 0.028409090909090908,
 0.1076923076923077,
 0.14285714285714285,
 0.015414258188824663,
 0.0718232044198895,
 0.0,
 0.07142857142857142,
 0.024390243902439025,
 0.008130081300813009,
 0.02608695652173913,
 0.11794871794871795,
 0.018867924528301886,
 0.038461538461538464,
 0.014705882352941176,
 0.11764705882352941,
 0.07058823529411765,
 0.0,
 0.08080808080808081,
 0.10638297872340426,
 0.04025423728813559,
 0.05384615384615385,
 0.07317073170731707,
 0.11538461538461539,
 0.13636363636363635,
 0.0,
 0.08604651162790698,
 0.14444444444444443]

Now, using the bestModel, we will check the results over the test data. Your result should be ~`0.0507`.  
Step below takes 1 minute to run. Uncomment last line if you wish to run and calculate model score. 

## Trying Some Artist Recommendations
Using the best model above, predict the top 5 artists for user `1059637` using the [recommendProducts](http://spark.apache.org/docs/1.5.2/api/python/pyspark.mllib.html#pyspark.mllib.recommendation.MatrixFactorizationModel.recommendProducts) function. Map the results (integer IDs) into the real artist name using `artistAlias`. Print the results. The output should look as follows:
```
Artist 0: My Chemical Romance
Artist 1: Something Corporate
Artist 2: Evanescence
Artist 3: Alanis Morissette
Artist 4: Counting Crows
```

In [10]:
# Find the top 5 artists for a particular user and list their names
# YOUR CODE GOES HERE
topFiveForUser = bestModel.recommendProducts(1059637, 5) #Get the user IDs of top 5 Artists for this user. 
for i in range(len(topFiveForUser)):
    print("Artist {} : {}".format(i,artistData.filter(lambda a:a[0] == str(topFiveForUser[i][1])).map(lambda a:a[1]).collect()[0]))

Artist 0 : Something Corporate
Artist 1 : My Chemical Romance
Artist 2 : Counting Crows
Artist 3 : U2
Artist 4 : Green Day
